In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
CFG = {
    "csv_path":        "tariffs.csv",   # must contain columns below
    "timestamp_col":   "timestamp",     # parse-able datetime
    "tariff_col":      "tariff",        # numeric price (¢/kWh, £/MWh, …)
    "tz":              "UTC",           # adjust if CSV is local time
    "battery_kwh":     13.5,            # usable energy
    "max_charge_kw":   5.0,
    "max_discharge_kw":5.0,
    "house_load_kw":   0.8,             # constant baseload for demo
    "eta_round":       0.95,            # round-trip battery efficiency
    "plot_hours":      168,             # how many future hours to plot
}

In [ ]:
def read_tariffs(path: str, ts_col: str, tariff_col: str, tz: str) -> pd.DataFrame:
    """Return sorted DataFrame with DatetimeIndex."""
    df = pd.read_csv(path, usecols=[ts_col, tariff_col])
    df[ts_col] = pd.to_datetime(df[ts_col])
    df = df.set_index(ts_col).tz_localize(tz, ambiguous='infer')
    return df.sort_index()

In [ ]:
def weekly_benchmark(df: pd.DataFrame, tariff_col: str) -> float:
    """Mean tariff of the last complete 168-hour block."""
    last_week = df.tail(7*24)
    if len(last_week) < 7*24:
        raise ValueError("Need ≥ 168 hourly records in CSV.")
    return float(last_week[tariff_col].mean())

In [ ]:
def plan_hour(tariff: float, benchmark: float,
              soc_prev: float, bat_kwh: float,
              max_c: float, max_d: float,
              load: float, eta: float) -> dict:
    """
    Returns {
        grid_kw: float,      # >0 import
        battery_kw: float,   # >0 charge, <0 discharge
        soc_new:  float,     # 0–1
        mode:     str        # "charge" | "discharge"
    }
    """
    if tariff >= benchmark:                       # DISCHARGE
        wanted_d = min(load/eta, max_d, soc_prev*bat_kwh)
        grid_kw    = max(0.0, load - wanted_d*eta)
        battery_kw = -wanted_d
        soc_new    = soc_prev - wanted_d/bat_kwh
        mode       = "discharge"
    else:                                         # CHARGE
        max_e = min(max_c, (1.0 - soc_prev)*bat_kwh)
        battery_kw = max_e
        grid_kw    = load + battery_kw
        soc_new    = soc_prev + battery_kw*eta/bat_kwh
        mode       = "charge"

    return dict(grid_kw=grid_kw,
                battery_kw=battery_kw,
                soc_new=np.clip(soc_new, 0, 1),
                mode=mode)

In [ ]:
def simulate(df_future: pd.DataFrame, benchmark: float, cfg: dict) -> pd.DataFrame:
    """df_future must have DatetimeIndex and tariff column."""
    records = []
    soc = 0.5                                          # start 50 %
    for ts, row in df_future.iterrows():
        res = plan_hour(tariff   = row[cfg["tariff_col"]],
                        benchmark= benchmark,
                        soc_prev = soc,
                        bat_kwh  = cfg["battery_kwh"],
                        max_c    = cfg["max_charge_kw"],
                        max_d    = cfg["max_discharge_kw"],
                        load     = cfg["house_load_kw"],
                        eta      = cfg["eta_round"])
        records.append({
            "timestamp" : ts,
            "tariff"    : row[cfg["tariff_col"]],
            "benchmark" : benchmark,
            "grid_kw"   : res["grid_kw"],
            "battery_kw": res["battery_kw"],
            "soc"       : res["soc_new"],
            "mode"      : res["mode"],
        })
        soc = res["soc_new"]
    return pd.DataFrame(records).set_index("timestamp")

In [ ]:
# 7-a load data
df_all = read_tariffs(CFG["csv_path"], CFG["timestamp_col"],
                      CFG["tariff_col"], CFG["tz"])

# 7-b compute benchmark
benchmark = weekly_benchmark(df_all, CFG["tariff_col"])
print(f"Past-week average tariff = {benchmark:.2f}")

# 7-c split future
now = pd.Timestamp.now(tz=CFG["tz"])
df_future = df_all[df_all.index >= now].copy()
if df_future.empty:
    raise RuntimeError("CSV has no future records – cannot schedule.")

# 7-d run
schedule = simulate(df_future, benchmark, CFG)
print("Simulation finished – first 5 rows:")
display(schedule.head())

In [ ]:
def plot_tariff(sched: pd.DataFrame, horizon: int):
    """Plot tariff vs benchmark for chosen horizon (hours)."""
    plt.figure(figsize=(14,4))
    sub = sched.head(horizon)
    plt.plot(sub.index, sub["tariff"], label="Hourly tariff")
    plt.axhline(sub["benchmark"].iloc[0], color="black", ls="--", lw=1.5,
                label=f"Benchmark ({sub['benchmark'].iloc[0]:.2f})")
    plt.ylabel("Tariff")
    plt.title("Electricity tariff vs weekly-average benchmark")
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_tariff(schedule, CFG["plot_hours"])

In [ ]:
def plot_power_soc(sched: pd.DataFrame, horizon: int):
    sub = sched.head(horizon)
    fig, ax = plt.subplots(2,1,figsize=(14,6), sharex=True)

    # power
    ax[0].step(sub.index, sub["grid_kw"], where="post", label="Grid import")
    ax[0].step(sub.index, -sub["battery_kw"], where="post", label="Battery discharge")
    ax[0].set_ylabel("kW")
    ax[0].legend()

    # SOC
    ax[1].step(sub.index, sub["soc"], where="post", color="green")
    ax[1].set_ylabel("SOC (fraction)")
    ax[1].set_ylim(0,1)

    plt.suptitle("Battery operation & state-of-charge")
    plt.tight_layout()
    plt.show()

plot_power_soc(schedule, CFG["plot_hours"])

In [ ]:
schedule.to_csv("battery_schedule.csv")
print("Schedule saved → battery_schedule.csv")